In [3]:
# Problem 1.1 — Reverse a directed graph in O(V + E)

def reverse_graph(adj):
    rev = {u: [] for u in adj}
    for u in adj:
        for v in adj[u]:
            rev[v].append(u)     # reverse edge
    return rev

# Example
G = {
    "A": ["B", "C"],
    "B": ["C"],
    "C": []
}

print("Original G:", G)
print("Reversed:", reverse_graph(G))


Original G: {'A': ['B', 'C'], 'B': ['C'], 'C': []}
Reversed: {'A': [], 'B': ['A'], 'C': ['A', 'B']}


In [4]:
# Problem 1.2 — Tarjan SCC (O(V + E))

def tarjan_scc(adj):
    index = 0
    stack = []
    indices = {}
    low = {}
    onstack = set()
    comps = []

    def dfs(v):
        nonlocal index
        indices[v] = index
        low[v] = index
        index += 1
        stack.append(v)
        onstack.add(v)

        for w in adj[v]:
            if w not in indices:
                dfs(w)
                low[v] = min(low[v], low[w])
            elif w in onstack:
                low[v] = min(low[v], indices[w])

        # root of SCC
        if low[v] == indices[v]:
            comp = []
            while True:
                w = stack.pop()
                onstack.remove(w)
                comp.append(w)
                if w == v:
                    break
            comps.append(comp)

    for v in adj:
        if v not in indices:
            dfs(v)

    return comps


In [8]:

from collections import defaultdict

def condensation_graph(adj):
    sccs = tarjan_scc(adj)

    # map vertex → component ID
    comp_id = {}
    for i, comp in enumerate(sccs):
        for v in comp:
            comp_id[v] = i

    # IMPORTANT: create all SCC nodes, even if no outgoing edges
    dag = {i: [] for i in range(len(sccs))}

    # build edges between SCCs
    for u in adj:
        for v in adj[u]:
            cu, cv = comp_id[u], comp_id[v]
            if cu != cv and cv not in dag[cu]:
                dag[cu].append(cv)

    return sccs, dag


In [9]:

def has_cycle(adj):
    color = {u: 0 for u in adj}   # 0=white, 1=grey, 2=black

    def dfs(v):
        color[v] = 1
        for w in adj[v]:
            # w guaranteed to be in adj if condensation_graph was built correctly
            if color[w] == 1:
                return True
            if color[w] == 0 and dfs(w):
                return True
        color[v] = 2
        return False

    for v in adj:
        if color[v] == 0:
            if dfs(v):
                return True
    return False


In [10]:

def prove_scc_graph_is_DAG(adj):
    sccs, dag = condensation_graph(adj)

    print("SCCs:", sccs)
    print("Condensed edges:", dag)

    if not dag:
        print("\nCondensation graph trivial → DAG.")
        return

    if has_cycle(dag):
        print("\nCycle detected! Showing contradiction...\n")
        print("If there is a cycle among SCCs, then:")
        print("  • All vertices in those SCCs are mutually reachable;")
        print("  • Tarjan's algorithm would merge them into ONE SCC;")
        print("  • So having different SCCs on a cycle contradicts the definition.")
        print("\nTherefore, condensation graph cannot have cycles in reality.")
    else:
        print("\nNo cycle → condensation graph is a DAG (as expected).")

# Example graph (fixed: C has both edges A and D)
G_test = {
    "A": ["B"],
    "B": ["C"],
    "C": ["A", "D"],  # SCC {A,B,C} plus edge to D
    "D": []
}

prove_scc_graph_is_DAG(G_test)


SCCs: [['D'], ['C', 'B', 'A']]
Condensed edges: {0: [], 1: [0]}

No cycle → condensation graph is a DAG (as expected).


In [11]:


def compare_scc_partitions(adj):
    rev = reverse_graph(adj)

    scc_G = sorted([sorted(c) for c in tarjan_scc(adj)])
    scc_rev = sorted([sorted(c) for c in tarjan_scc(rev)])

    print("SCC(G)      =", scc_G)
    print("SCC(rev(G)) =", scc_rev)
    print("Partitions identical? →", scc_G == scc_rev)

compare_scc_partitions(G_test)


SCC(G)      = [['A', 'B', 'C'], ['D']]
SCC(rev(G)) = [['A', 'B', 'C'], ['D']]
Partitions identical? → True


In [12]:

def check_reversal_of_scc_graph(adj):
    scc_G, dag_G = condensation_graph(adj)
    scc_rev, dag_rev = condensation_graph(reverse_graph(adj))

    print("condensed G:", dag_G)
    print("condensed rev(G):", dag_rev)

    # Reverse DAG of G
    rev_of_dag_G = {u: [] for u in dag_G}
    for u in dag_G:
        for v in dag_G[u]:
            rev_of_dag_G.setdefault(v, []).append(u)

    print("reversed condensed G:", rev_of_dag_G)
    print("Equal to condensed rev(G)? →", dag_rev == rev_of_dag_G)

check_reversal_of_scc_graph(G_test)


condensed G: {0: [], 1: [0]}
condensed rev(G): {0: [], 1: [0]}
reversed condensed G: {0: [1], 1: []}
Equal to condensed rev(G)? → False


In [13]:

from collections import deque

def reachable(adj, start):
    visited = set([start])
    q = deque([start])
    while q:
        u = q.popleft()
        for v in adj[u]:
            if v not in visited:
                visited.add(v)
                q.append(v)
    return visited


In [14]:

def verify_reachability_equivalence(adj):
    sccs, dag = condensation_graph(adj)

    comp = {}
    for i, comp_list in enumerate(sccs):
        for v in comp_list:
            comp[v] = i

    vertices = list(adj.keys())

    print("\nChecking equivalence for all pairs (u, v):\n")
    for u in vertices:
        for v in vertices:
            # in original graph
            r1 = v in reachable(adj, u)
            # in SCC graph
            r2 = comp[v] in reachable(dag, comp[u]) if dag else True

            print(f"{u} → {v}:", r1, "| SCC:", r2, "| OK?", r1 == r2)

verify_reachability_equivalence(G_test)



Checking equivalence for all pairs (u, v):

A → A: True | SCC: True | OK? True
A → B: True | SCC: True | OK? True
A → C: True | SCC: True | OK? True
A → D: True | SCC: True | OK? True
B → A: True | SCC: True | OK? True
B → B: True | SCC: True | OK? True
B → C: True | SCC: True | OK? True
B → D: True | SCC: True | OK? True
C → A: True | SCC: True | OK? True
C → B: True | SCC: True | OK? True
C → C: True | SCC: True | OK? True
C → D: True | SCC: True | OK? True
D → A: False | SCC: False | OK? True
D → B: False | SCC: False | OK? True
D → C: False | SCC: False | OK? True
D → D: True | SCC: True | OK? True


In [15]:
# Problem 2.1 — Compute degrees

def degrees(adj):
    indeg = {u: 0 for u in adj}
    outdeg = {u: len(adj[u]) for u in adj}
    for u in adj:
        for v in adj[u]:
            indeg[v] += 1
    return indeg, outdeg


In [16]:
#  Check Euler tour condition

def has_euler_tour(adj):
    indeg, outdeg = degrees(adj)
    for v in adj:
        if indeg[v] != outdeg[v]:
            return False
    return True

# Example
G_euler = {
    "A": ["B"],
    "B": ["C"],
    "C": ["A"]   # cycle → Eulerian
}

print("Euler tour exists?", has_euler_tour(G_euler))


Euler tour exists? True


In [17]:
# Problem 2.2 Hierholzer algorithm

def euler_tour(adj):
    if not has_euler_tour(adj):
        return None

    graph = {u: list(adj[u]) for u in adj}
    start = next(iter(graph))
    stack = [start]
    tour = []

    while stack:
        v = stack[-1]
        if graph[v]:
            u = graph[v].pop()
            stack.append(u)
        else:
            tour.append(stack.pop())

    return tour[::-1]   # reverse


print("Euler tour:", euler_tour(G_euler))


Euler tour: ['A', 'B', 'C', 'A']


In [18]:
# Problem 3 — Course DAG

courses = ["A", "B", "C", "D", "E", "F", "G"]

edges = [
    ("A", "B"),
    ("A", "C"),
    ("B", "C"),
    ("B", "D"),
    ("C", "E"),
    ("D", "E"),
    ("D", "F"),
    ("G", "F"),
    ("G", "E"),
]

adj = {c: [] for c in courses}
indeg = {c: 0 for c in courses}

for u, v in edges:
    adj[u].append(v)
    indeg[v] += 1


In [19]:
#Topological sort with optional preferred start

import heapq
import copy

def topo_sort(adj, indeg, preferred=None):
    indeg = copy.deepcopy(indeg)

    pq = []
    for v in adj:
        if indeg[v] == 0:
            heapq.heappush(pq, v)

    order = []

    # Try using preferred start
    if preferred in pq:
        pq.remove(preferred)
        heapq.heapify(pq)
        order.append(preferred)
        for w in adj[preferred]:
            indeg[w] -= 1
            if indeg[w] == 0:
                heapq.heappush(pq, w)

    while pq:
        v = heapq.heappop(pq)
        order.append(v)
        for w in adj[v]:
            indeg[w] -= 1
            if indeg[w] == 0:
                heapq.heappush(pq, w)

    return order

print("Topological sort from A:", topo_sort(adj, indeg, preferred="A"))
print("Topological sort from G:", topo_sort(adj, indeg, preferred="G"))


Topological sort from A: ['A', 'B', 'C', 'D', 'G', 'E', 'F']
Topological sort from G: ['G', 'A', 'B', 'C', 'D', 'E', 'F']
